In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [1]:
cd ~/gscratch/athena_simulation_data/log/

/global/scratch2/sd/dorislee/athena_simulation_data/log


In [3]:
import sys
sys.path.remove('/global/homes/d/dorislee/.local/lib/python2.7/site-packages/yt-3.2.dev0-py2.7-linux-x86_64.egg')
sys.path.remove('/global/homes/d/dorislee/.local/lib/python2.7/site-packages')
sys.path.append('/global/homes/d/dorislee/mypythonpackages')

In [4]:
import h5py
import glob

In [5]:
filename="log.out1.00002.ath" #just read the athena hdf5 file not the xdmf file
f = h5py.File(filename, 'r+')

#### This simulation was 96x64x128 where I defined each meshblock to be 32x32x32 so that I have 24 meshblock one of each processor , making use of one full NUMA node on Hopper

In [6]:
for i in f.keys():print i

MeshBlock0
MeshBlock1
MeshBlock10
MeshBlock11
MeshBlock12
MeshBlock13
MeshBlock14
MeshBlock15
MeshBlock16
MeshBlock17
MeshBlock18
MeshBlock19
MeshBlock2
MeshBlock20
MeshBlock21
MeshBlock22
MeshBlock23
MeshBlock3
MeshBlock4
MeshBlock5
MeshBlock6
MeshBlock7
MeshBlock8
MeshBlock9


In [7]:
meshblock =f["MeshBlock0"]

In [8]:
meshblock.keys()

[u'press', u'rho', u'vel1', u'vel2', u'vel3', u'x1f', u'x2f', u'x3f']

In [9]:
x1f = meshblock["x1f"].value
print x1f
# plt.plot(x1f,'o') #linearly sampled series of r values in a meshblock

[ 0.2         0.20895773  0.21818419  0.22768743  0.2374758   0.24755779
  0.25794226  0.26863825  0.27965513  0.29100251  0.3026903   0.31472874
  0.32712832  0.3398999   0.35305464  0.366604    0.38055986  0.39493439
  0.40974012  0.42499006  0.44069749  0.45687616  0.47354016  0.49070409
  0.50838292  0.52659214  0.54534763  0.56466579  0.58456349  0.60505813
  0.6261676   0.64791036  0.67030543]


$$r_i = \frac{(r_+ + r_-)}{2}$$

In [10]:
ri = (x1f[0]+x1f[-1])/2.
print ri

0.435152709484


In [11]:
dri = abs(x1f[0]-x1f[-1])
print dri

0.470305


Computing the r,$\theta$ surface density ~~over the whole $\phi_i$ slice~~ per cell
$$\Delta Si,j,k =\int\int r drd\theta = r_i\Delta r_i\Delta \theta_j$$

total of 64 meshblocks spanning from $\theta$=0.32 to 2.79 radian and 32 zones per Meshblock

In [12]:
(2.79252680319-0.34906585039)/64.*32

1.2217304763999999

In [13]:
x2f = meshblock["x2f"].value
print x2f

[ 0.34906584  0.38724494  0.42542401  0.46360308  0.50178218  0.53996122
  0.57814032  0.61631942  0.65449846  0.69267756  0.7308566   0.7690357
  0.8072148   0.84539384  0.88357294  0.92175204  0.95993108  0.99811018
  1.03628922  1.07446837  1.11264741  1.15082645  1.18900549  1.22718465
  1.26536369  1.30354273  1.34172189  1.37990093  1.41807997  1.45625913
  1.49443817  1.53261721  1.57079637]


In [14]:
dthetai = abs(x2f[0]-x2f[-1])
print dthetai

1.22173


Checked that these are the same

In [15]:
dS = ri*dri*dthetai
print dS

0.250032867955


$$D(\phi) = \text{surface density} = \Sigma\Sigma\rho\Delta S_\phi $$

In [23]:
density= meshblock["rho"].value#density for each meshblock
N_r = density.shape[0]
N_theta=density.shape[1]
N_phi= density.shape[2]

In [17]:
# density[:,:,phi-1].shape

Summing over a $r,\theta$ slice of density: 

In [25]:
for phi_i in np.arange(N_phi):
    print sum(density[:,:,phi_i-1]) #D(phi)

0.102834
0.0993134
0.0989731
0.0988605
0.0988546
0.0989099
0.0989884
0.0990942
0.0993259
0.0997008
0.10025
0.100857
0.101339
0.101665
0.101884
0.102119
0.102376
0.102578
0.102644
0.102569
0.102463
0.102397
0.102446
0.102603
0.102819
0.103021
0.103178
0.103305
0.10344
0.1036
0.103727
0.103767


approx $\rho \approx <\rho>$

In [19]:
density_avrg =  mean(density)
print density_avrg 

0.000125047


pretty low, but makes sense because this is outside the torus so gas pressure is suppose to be low (e-5)..

Computing $D(\phi)$

In [36]:
NUM_MESHBLOCK=24
# N=0
D_lst = []
for i in np.arange(NUM_MESHBLOCK):
    meshblock =f["MeshBlock{}".format(i)]
    x1f = meshblock["x1f"].value
    rg =  mean(density)
    ri = (x1f[0]+x1f[-1])/2.
    dri = abs(x1f[0]-x1f[-1])
    x2f = meshblock["x2f"].value
    dthetai = abs(x2f[0]-x2f[-1])
    dS = ri*dri*dthetai
    #S_phi should be constant for all phi
    D=[]
    for phi_i in np.arange(N_phi):
        D.append(sum(density[:,:,phi_i-1])*dS)
    #print sum(D)
#     D +=density_avrg*dS
#     N+=1
# print N
    print "D(phi) in MeshBlock{}: {}\n".format(i,D) 
    D_lst.append(D)

D(phi) in MeshBlock0: [0.025711988286840654, 0.024831623714298822, 0.02474651944595907, 0.024718371177536565, 0.024716893905739144, 0.024730714686905693, 0.024750345822090097, 0.024776817489481479, 0.024834742192216711, 0.02492847164141657, 0.025065688395583666, 0.025217508344731836, 0.025338169595161619, 0.025419715743535326, 0.025474272341088035, 0.025533184375541595, 0.02559730505053184, 0.025647977522362451, 0.02566430016484544, 0.025645572531327807, 0.025619149299077326, 0.025602610561350322, 0.025614892595348309, 0.025654156728607151, 0.025708109749788658, 0.025758702117347786, 0.025797860065874655, 0.025829599986283831, 0.025863288489669175, 0.025903519462855965, 0.02593518114188369, 0.02594507308796731]

D(phi) in MeshBlock1: [0.19412541520083798, 0.18747866597762611, 0.18683612907061944, 0.18662360974952516, 0.18661245635299123, 0.18671680319900039, 0.18686501819606799, 0.18706487918566095, 0.18750221047421856, 0.18820986746439308, 0.18924585344408615, 0.19039209349115563, 0.1

In [22]:
len(D)

32

In [41]:
oneD_density_in_phi = np.array(D_lst).flatten()

In [43]:
len(oneD_density_in_phi)

768

32 meshblocks in phi direction(spanning 128 zones) and 32 cells in each meshblock

In [47]:
32*24

768